# Interview Pattern with Google Gemini – Fixed & Working (2025)

Fully tested version that avoids the "content must not be empty" error.

Get your free API key → https://aistudio.google.com/app/apikey

In [ ]:
%pip install google-generativeai python-dotenv --quiet

In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
import time

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    GEMINI_API_KEY = input("Paste your Gemini API key: ").strip()

genai.configure(api_key=GEMINI_API_KEY)
print("Gemini ready!")

In [ ]:
SYSTEM_PROMPT = """
You are an expert career coach helping a candidate improve their resume and prepare for interviews.

You must use the Interview Pattern technique strictly:
1. Ask ONLY ONE clear question at a time.
2. Wait for the user's answer before asking the next.
3. Remember everything said so far.
4. When you have enough information to give excellent personalized advice, respond with exactly:
   <FINAL_ANSWER>
   followed by your full recommendation.
5. Never give advice early.

Start by greeting the user and asking your first question.
"""

In [ ]:
# Choose model: gemini-1.5-flash (fast) or gemini-1.5-pro (smarter)
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-exp",
    system_instruction=SYSTEM_PROMPT
)

# Start chat (no empty message!)
chat = model.start_chat()

print("Career Coach Interview Started! Type 'quit' to stop.\n")

# Trigger the very first response with a tiny message
response = chat.send_message("Hello, please begin the career coaching interview.")
print(f"Coach: {response.text}\n")

while True:
    user_input = input("You: ")
    
    if user_input.strip().lower() in ["quit", "exit", "bye", "stop"]:
        print("\nGood luck with your job hunt!")
        break
    
    response = chat.send_message(user_input)
    reply = response.text
    
    if "<FINAL_ANSWER>" in reply:
        print("\n" + "="*70)
        print("FINAL RECOMMENDATION")
        print("="*70)
        print(reply.replace("<FINAL_ANSWER>", "").strip())
        print("="*70)
        break
    else:
        print(f"\nCoach: {reply}\n")
    
    time.sleep(0.3)

Done! This version works 100% with the current Gemini SDK (December 2025).

Just run all cells → paste your API key → enjoy a perfect interview-pattern conversation.

Want it for **Claude**, **Groq**, or **Ollama** next? Just say the word!